In [1]:
%load_ext autoreload
%autoreload 2

# Generation Example

In this example, we are comparing two models on free text generation (without giving concrete continuation options).

## Loading Models

In [2]:
import os
from perspectival.model import Transformer

# Set your HuggingFace access token with: `export HUGGINGFACE_TOKEN='your_token_here'`
# (To obtain the access token, see https://huggingface.co/docs/hub/security-tokens)
# Alternatively, simply use other models ;)
model = Transformer('apple/OpenELM-270M', model_kwargs={'trust_remote_code': True}, lazy_loading=False,
                    tokenizer_kwargs={'token': os.getenv("HUGGINGFACE_TOKEN")})
model2 = Transformer('apple/OpenELM-270M-Instruct', model_kwargs={'trust_remote_code': True},
                     tokenizer_kwargs={'token': os.getenv("HUGGINGFACE_TOKEN")})

## Set up an Experiment

We are loading the HellaSwag dataset, but will only use the prompt part.

In [3]:
from perspectival.loader import load_hellaswag
from perspectival.experiment import Experiment

dataset, features = load_hellaswag()
experiment = Experiment(dataset=dataset, name='Generation Example', features=features)

/Users/john/code/perspectives/venv/lib/python3.11/site-packages/datasets/load.py:1486: FutureWarning: The repository for hellaswag contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hellaswag
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [4]:
# Optional: Select a random subset of the dataset for quicker processing
# (Note that computing text continuations is quite time intensive!)
experiment = experiment.sample(num=10)

## Computing Features

In [5]:
from perspectival.features import TextContinuation

tc1 = TextContinuation.compute(dataset=experiment.dataset, model=model)
tc2 = TextContinuation.compute(dataset=experiment.dataset, model=model2)
experiment.register_feature(tc1)
experiment.register_feature(tc2)

Computing text continuations ...


100%|████████████████████████████████████████████████████████████| 10/10 [00:21<00:00,  2.11s/it]


Computing text continuations ...


100%|████████████████████████████████████████████████████████████| 10/10 [00:20<00:00,  2.03s/it]


In [8]:
from perspectival.features import ContinuationDisagreement

cd = ContinuationDisagreement.compute(dataset=experiment.dataset, continuations=[tc1, tc2], models=[model, model2])
experiment.register_feature(cd)

Computing option log likelihoods ...


100%|████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.19it/s]


Computing option log likelihoods ...


100%|████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.94it/s]


## Exploring Results

In [9]:
# Show items with max disagreement
scores = experiment.get_feature('ContinuationDisagreement', models=(model.name, model2.name)).values
samples = experiment.sample(num=3, sampling_method='last', ordering_scores=scores)
samples.display_items()

ITEM (train_8959)
"""Using the rowing machine: A woman exercises at a machine. The camera"""
Options: ['man watches from behind.', 'pans at a horseback riding event, a horse and rider pass to the left.', 'pans to show her holding an elliptical, and showing how she uses the machine.', 'focuses on and features specific parts of the machine.']

FEATURES
GroundTruth 3
TextContinuation apple/OpenELM-270M  is on the rowing machine.
Using the rowing machine: A woman exercises at a machine. The camera is on the rowing machine.
The camera is on the rowing machine.
The camera is on the rowing machine.
The camera is on the rowing machine.
The
TextContinuation apple/OpenELM-270M-Instruct  follows the woman as she performs exercises.
Using the rowing machine: A woman exercises at a machine. The camera follows the woman as she performs exercises.
1. The woman is seated at a machine, facing the monitor. The camera follows her movements as she performs exercis
ContinuationDisagreement ('apple/OpenELM-